# 📘 代理架构 15：RLHF（人类反馈强化学习）

欢迎来到我们系列的第十五个notebook。今天，我们探索**RLHF**架构，这是一种使用人类反馈来改进代理行为的模式。

RLHF允许代理从人类偏好中学习，使其输出更符合人类期望和价值观。

### 定义
一个**Self-Improvement Loop** 是代理架构 其中代理's输出is 评估，通过itself or通过another 代理,和这个evaluation 是used 作为feedback来generate 一个revised, higher-quality output. 当这些反馈被存储并用于随时间改进代理的基线性能时,它becomes 一个form的continual learning.

### 高级工作流程 (Self-Refine)

1. **Generate Initial 输出:** primary 代理 produces 一个first version的solution (the "draft").
2. **Critique 输出:** 一个critic 代理 (or primary 代理在一个"critique mode") evaluates draft against 一个set的predefined criteria或一个general rubric.
3. **Decision:** 系统 检查 如果critique 是positive enough来accept output. 
4. **Revise (Loop):** 如果输出is不accepted, original draft *and* critic's feedback 是passed back来primary 代理, which 是instructed来generate 一个revised version 那个addresses feedback.
5. **Accept:** Once输出meets质量standard, loop terminates,和final version 被重新turned.

### 何时使用/应用场景
* **High-质量 内容 Generation:** 为任务 其中一个generic first draft 是insufficient, such 作为writing legal documents, detailed technical reports,或persuasive marketing copy.
* **Continual Learning & Personalization:** 一个代理 那个learns 一个user's preferences通过generating responses, getting implicit或explicit feedback,和refining它的internal 策略为next interaction.
* **Complex Problem Solving:** 一个代理 可以propose 一个plan, critique it为flaws或inefficiencies,和then revise计划before execution.

### 优点和缺点
* **优点：**
 * **Dramatically Increases 输出 质量:** Iterative refinement consistently produces better结果than single-shot generation.
 * **Enables Continual Learning:** Provides 一个framework为代理来get更好的over time, adapting来new information或feedback.
* **缺点：**
 * **Risk的Reinforcing Biases:** 如果 critic 代理 有flawed logic或biases, 系统 can获取stuck在一个loop 那个reinforces它的own mistakes.
 * **Computationally Expensive:** iterative nature means multiple LLM calls per task, increasing cost和latency.

## 阶段0：基础与设置

Standard setup的库和environment variables.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown from rich.panel import Panel

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Self-Improvement Loop (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：定义核心组件 (Generator, Critic, Reviser)

Our 系统 needs distinct 角色提供动力. 我们将定义personas和structured outputs用于我们的`JuniorCopywriter` (the generator)和our `SeniorEditor` (the critic). `Reviser` isn't 一个new 代理，但rather 一个different mode的invoking generator, armed与feedback.

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# --- Pydantic ModelsforStructured data ---
class MarketingEmail(BaseModel):
 """Represents 一个marketing email draft."""
 subject: str = Field(description="A catchyandconcise subject lineforemail.")
 body: str = Field(description="full body text的email, writteninmarkdown.")

class Critique(BaseModel):
 """A structured critique的marketing email draft."""
 score: int = Field(description="Overall质量scorefrom1 (poor)来10 (excellent).")
 feedback_points: List[str] = Field(description="A bulleted list的specific, actionable feedback pointsforimprovement.")
 is_approved: bool = Field(description="A boolean indicating if draft is approved (score >= 8). 这is redundantwithscore但usefulforrouting.")

# --- 1. Generator: Junior Copywriter ---
def get_generator_chain():
 prompt = ChatPromptTemplate.from_messages([
 ("system", "你is一个junior marketing copywriter. 你的任务isto write 一个first draft的marketing email 基于用户的请求. Be creative，但focusingetting coremessagesacross."),
 ("human", "Write 一个marketing email 关于following topic:\n\n{request}")
 ])
 returnprompt | llm.with_structured_output(MarketingEmail)

# --- 2. Critic: Senior Editor ---
def get_critic_chain():
 prompt = ChatPromptTemplate.from_messages([
 ("system", """你is一个senior marketing editorandbrand manager. 你的工作isto critique 一个email draft writtenpass一个junior copywriter. 
 Evaluate draft against following criteria:
 1. **Catchy 主题：** Is subject line engagingandlikely来get opened?
 2. **Clarity & Persuasiveness:** Is body text clear, compelling,andpersuasive?
 3. **Strong 调用-to-Action (CTA):** Is there 一个clear, single actionforuser来take?
 4. **Brand Voice:** Is tone professional yet 方法able?
 Provide 一个scorefrom1-10. A score的8orhigher means draft is approvedforsending. Provide specific, actionable feedback来help writer improve."""
 ),
 ("human", "请critique following email draft:\n\n**主题：** {subject}\n\n**正文：**\n{body}")
 ])
 returnprompt | llm.with_structured_output(Critique)

# --- 3. Reviser (Generatorin'Revise' Mode) ---
def get_reviser_chain():
 prompt = ChatPromptTemplate.from_messages([
 ("system", "你isjunior marketing copywriter who wrote original draft. You have just received feedbackfromyour senior editor. 你的任务isto carefully revise your draft来address every single point的feedback. Produce 一个new, improved version的email."),
 ("human", "Original 请求: {request}\n\nHere is your original draft:\n**主题：** {original_subject}\n**正文：**\n{original_body}\n\nHere is feedbackfromyour editor:\n{feedback}\n\nPlease提供revised email.")
 ])
 returnprompt | llm.with_structured_output(MarketingEmail)

print("GeneratorandCritic components defined successfully.")

GeneratorandCritic components 定义成功.


## 阶段2： 构建 Self-Refinement Loop与Lang图

Now 我们将 construct graph 那个automates `Generate -> Critique -> Revise` loop.状态will追踪draft, critique,和number的revisions. 一个conditional edge will检查critic's 评分来decide whether来exit loop或continue与another revision.

In [4]:
# LangGraph State
class AgentState(TypedDict):
 user_request: str
 draft_email: Optional[MarketingEmail]
 critique: Optional[Critique]
 revision_number: int

# Graph Nodes
def generate_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("📝 初级文案正in生成 初始草稿.", title="[yellow]步骤：生成[/yellow]", border_style="yellow"))
 chain = get_generator_chain()
 draft = chain.invoke({"request": state['user_request']})
 console.print(Panel(f"[bold]主题：[/bold] {draft.subject}\n\n{draft.body}", title="Draft 1"))
 return{"draft_email": draft, "revision_number": 1}

def critique_node(state: AgentState) -> Dict[str, Any]:
 title = f"[yellow]步骤：评审 (Revision #{state['revision_number']})[/yellow]"
 console.print(Panel(f"🧐 Senior Editor is critiquing draft #{state['revision_number']}.", title=title, border_style="yellow"))
 chain = get_critic_chain()
 critique_result = chain.invoke(state['draft_email'].dict())
 feedback_text = "\n- ".join(critique_result.feedback_points)
 console.print(Panel(f"[bold]score:[/bold] {critique_result.score}/10\n[bold]反馈：[/bold]\n- {feedback_text}", title="Critique result"))
 return{"critique": critique_result}

def revise_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("✍️ Junior Copywriter is revising draft basedinfeedback.", title="[yellow]步骤: Revise[/yellow]", border_style="yellow"))
 chain = get_reviser_chain()
 feedback_str = "\n- ".join(state['critique'].feedback_points)
 revised_draft = chain.invoke({
 "request": state['user_request'],
 "original_subject": state['draft_email'].subject,
 "original_body": state['draft_email'].body,
 "feedback": feedback_str,
 })
 console.print(Panel(f"[bold]主题：[/bold] {revised_draft.subject}\n\n{revised_draft.body}", title=f"Draft {state['revision_number'] + 1}"))
 return{"draft_email": revised_draft, "revision_number": state['revision_number'] + 1}

# Conditional 边
def should_continue(state: AgentState) -> str:
 console.print(Panel("⚖️ Decision Point: Does draft meet质量standards?", title="[yellow]步骤: Decide[/yellow]", border_style="yellow"))
 if state['critique'].is_approved:
 console.print("[green]结论: Critique APPROVED! Finishing 过程的核心。[/green]")
 return "end"
 if state['revision_number'] >= 3: # Set 一个max revision limit
 console.print("[red]结论: Max revisions reached. Finishingwithlast draft.[/red]")
 return "end"
 else:
 console.print("[yellow]结论: Critique requires revision. Looping back.[/yellow]")
 return "continue"

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("generate", generate_node)
workflow.add_node("critique", critique_node)
workflow.add_node("revise", revise_node)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "critique")
workflow.add_conditional_edges(
 "critique",
 should_continue,
 {"continue": "revise", "end": END}
)
workflow.add_edge("revise", "critique")

self_refine_agent = workflow.compile()
print("Self-Refinement 代理 Graph编译成功.")

Self-Refinement 代理 图编译成功.


## 阶段3： 自我改进循环的演示

让我们运行代理并观察迭代改进过程的核心。 我们将要求它为新的AI产品编写电子邮件，并观察它如何生成、被批评并修订自己的工作，直到达到质量标准.

In [5]:
def run_agent(request: str):
 initial_state = {"user_request": request}
 # stream() 允许 us来see intermediate steps
 final_state = None
 forstepinself_refine_agent.stream(initial_state):
 # finalstateis one just before END is called
 if END notinstep:
 final_state = list(step.values())[0]
 returnfinal_state

request = "Write 一个marketing email announcing our新的revolutionary AI-powereddataanalytics platform, 'InsightSphere'."
console.print(f"--- 🚀 启动 Self-Refinement 过程 ---")
final_result = run_agent(request)

# Display final, approved result
console.print("\n--- 最终批准的邮件 ---")
final_email = final_result['draft_email']
final_critique = final_result['critique']
email_panel = Panel(
 f"[bold]主题：[/bold] {final_email.subject}\n\n---\n\n{final_email.body}",
 title="[bold green]Approved Email[/bold green]",
 subtitle=f"[green]最终 score: {final_critique.score}/10[/green]",
 border_style="green"
)
console.print(email_panel)

--- 🚀 启动 Self-Refinement 过程 ---


 步骤：生成 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 初级文案正在生成 initial ┃
┃ draft. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 草稿 1 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 主题： New Product Announcement ┃
┃ ┃
┃ Hello, ┃
┃ ┃
┃ 我们是 happy来announce our新的product, InsightSphere. It's 一个┃
┃ AI-powered数据analytics platform. It 可以help you分析your ┃
┃ data. ┃
┃ ┃
┃ Click here来learn more. ┃
┃ ┃
┃ Thanks, ┃
┃ Team ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 步骤: 批评 (Revision #1) 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🧐 Senior Editor是critiquing draft #1. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 批评 结果 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 评分: 4/10 ┃
┃ Feedback: ┃
┃ - subject line 'New Product Announcement'是generic和 ┃
┃ uninspired. It needs来be more intriguing来grab attention. ┃
┃ - body是too simplistic和doesn't解释value ┃
┃ proposition. What问题does InsightSphere solve? Use more ┃
┃ persuasive language. ┃
┃ - 'Click here来learn more'是一个weak call-to-action. Be more ┃
┃ specific和创建urgency. ┃
┃ - tone是flat. It needs more energy和excitement来match ┃
┃ 一个'revolutionary' product. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

...

 批评 结果 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 评分: 9/10 ┃
┃ Feedback: ┃
┃ - Excellent work在revision. subject line是much ┃
┃ stronger和benefit-oriented. ┃
┃ - body now clearly articulates problem和presents ┃
┃ InsightSphere作为solution. language是persuasive和 ┃
┃ energetic. ┃
┃ - call-to-action是specific, clear,和creates 一个sense的 ┃
┃ exclusivity. ┃
┃ - This draft是approved和ready来send. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 步骤: Decide 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does ┃
┃ draft meet质量 ┃
┃ standards? ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

结论: 批评 APPROVED! Finishing 过程的核心。



--- 最终批准的邮件 ---


 Approved Email 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 主题： Unlock Your 数据's True Potential与InsightSphere ┃
┃ ┃
┃ --- ┃
┃ ┃
┃ Hi [Name], ┃
┃ ┃
┃ Are你struggling来turn massive datasets into actionable ┃
┃ insights? ┃
┃ ┃
┃ We're thrilled来introduce **InsightSphere**,我们的revolutionary ┃
┃新的AI-powered analytics platform. Stop guessing和start ┃
┃ knowing. InsightSphere surfaces hidden 模式s, predicts future ┃
┃ trends,和提供 clarity你need来make smarter, ┃
┃ data-driven decisions. ┃
┃ ┃
┃ Don't获取left behind. Be one的first来experience ┃
┃ future的business intelligence. ┃
┃ ┃
┃ **[Request 一个Personalized 演示 Today]** ┃
┃ ┃
┃ Best, ┃
┃ InsightSphere Team ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
 最终 评分: 9/10 

## 阶段4： Persistent Improvement - RLHF Analogy

self-refine loop improves quality为一个*single run*. 但how 做我们make 代理更好的*over time*? 我们can extend我们的架构来save high-quality, approved outputs和使用them 作for examples为future 任务. 这是对强化学习如何工作的实用的、应用级别的类比 Learning从Human/AI Feedback (RLHF) works.

我们将定义一个 简单的 in-memory `GoldStandardMemory`和一个new generator node 那个uses 这个memory来improve它的first draft.

In [6]:
class GoldStandardMemory:
 """A 简单的 in-memory storeforhigh-quality examples."""
 def __init__(self):
 self.examples: List[MarketingEmail] = []
 
 def add_example(self, email: MarketingEmail):
 self.examples.append(email)
 
 def get_formatted_examples(self) -> str:
 if not self.examples:
 return "No示例available yet."
 formatted = "\n\n---\n\n".join([
 f"示例 主题： {ex.subject}\n示例 正文：\n{ex.body}"
 forexinself.examples
 ])
 returnformatted

# Instantiate我们的persistent memory
gold_standard_memory = GoldStandardMemory()

# New generator node 那个uses memory
def generate_node_with_memory(state: AgentState) -> Dict[str, Any]:
 title = "[yellow]步骤：生成[/yellow]"
 console.print(Panel("📝 初级文案正in生成 初始草稿 (基于过去的成功经验).", title=title, border_style="yellow"))
 示例= gold_standard_memory.get_formatted_examples()
 
 prompt = ChatPromptTemplate.from_messages([
 ("system", "你is一个junior marketing copywriter. 你的任务isto write 一个first draft的marketing email 基于用户的请求. You 应该learnfromstyleandquality的past successful examples."),
 ("human", "Here are some examples的high-quality emails 那个were approvedpassyour editor:\n\n{examples}\n\nNow, write 一个marketing email 关于following topic:\n\n{request}")
 ])
 chain = prompt | llm.with_structured_output(MarketingEmail)
 draft = chain.invoke({"request": state['user_request'], "examples": examples})
 console.print(Panel(f"[bold]主题：[/bold] {draft.subject}\n\n{draft.body}", title=f"Draft {state.get('revision_number', 1)}"))
 return{"draft_email": draft, "revision_number": 1}

# Build新的graphwithmemory-enabled generator
workflow_with_memory = StateGraph(AgentState)
workflow_with_memory.add_node("generate", generate_node_with_memory)
workflow_with_memory.add_node("critique", critique_node)
workflow_with_memory.add_node("revise", revise_node)
workflow_with_memory.set_entry_point("generate")
workflow_with_memory.add_edge("generate", "critique")
workflow_with_memory.add_conditional_edges("critique", should_continue, {"continue": "revise", "end": END})
workflow_with_memory.add_edge("revise", "critique")
self_improving_agent = workflow_with_memory.compile()
print("Persistent memory components defined successfully.")

# --- DEMONSTRATION 的LONG-TERM IMPROVEMENT ---

# 1. Save我们的previously approved email来memory
console.print(Panel("high-quality, editor-approved emailfor'InsightSphere' has been saved. It 将now be used as 一个referenceforfuture generations.", title="[bold]🏆 Saving approved email来Gold Standard Memory[/bold]", border_style="magenta"))
gold_standard_memory.add_example(final_result['draft_email'])

# 2. Run 代理 againin一个NEW task
new_request = "Write 一个promotional emailfor our新的AI-powered CRM called 'Visionary'."
console.print("\n--- 🚀 启动 Self-Refinement 过程withMemory ---")
new_final_result = run_agent(new_request)

# 3. Display新结果. key thing来notice is if it gets approved faster.
console.print("\n--- 最终批准的邮件 (GeneratedwithMemory) ---")
new_final_email = new_final_result['draft_email']
new_critique = new_final_result['critique']
email_panel_2 = Panel(
 f"[bold]主题：[/bold] {new_final_email.subject}\n\n---\n\n{new_final_email.body}",
 title="[bold green]Approved Email[/bold green]",
 subtitle=f"[green]最终 score: {new_critique.score}/10[/green]",
 border_style="green"
)
console.print(email_panel_2)

Persistent memory components 定义成功.


 🏆 Saving approved email来Gold Standard Memory 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ high-quality, editor-approved email为'InsightSphere'有┃
┃ been saved. It 将now be used作为一个reference为future ┃
┃ generations. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


--- 🚀 启动 Self-Refinement 过程withMemory ---


 步骤：生成 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 Junior Copywriter是generating ┃
┃ initial draft (Informed通过Past ┃
┃ 成功es). ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 草稿 1 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 主题： Go 从数据来Decisions, Instantly,与Visionary ┃
┃ ┃
┃ Hi [Name], ┃
┃ ┃
┃ Is你的team drowning在data但starving为wisdom? ┃
┃ ┃
┃ Introducing **Visionary**,我们的groundbreaking AI-powered CRM ┃
┃ 那个doesn't just store customer information—it understands it. ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs,和flags at-risk accounts, empowering你的team来act ┃
┃ proactively. ┃
┃ ┃
┃ Ready来transform你的customer relationships? ┃
┃ ┃
┃ **[Schedule 一个15-Minute Live 演示]** ┃
┃ ┃
┃ All best, ┃
┃ Visionary Team ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

...

 批评 结果 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 评分: 9/10 ┃
┃ Feedback: ┃
┃ - 这是一个strong first draft 那个clearly learned从 ┃
┃ previous example. subject是benefit-driven和compelling. ┃
┃ - body 有效的ly uses 问题来hook reader和 ┃
┃ explains value proposition clearly. ┃
┃ - call-to-action是specific和有效的. ┃
┃ - brand voice是spot on. 这是approved. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 步骤: Decide 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does ┃
┃ draft meet质量 ┃
┃ standards? ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

结论: 批评 APPROVED! Finishing 过程的核心。



--- 最终批准的邮件 (Generated与Memory) ---


 Approved Email 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 主题： Go 从数据来Decisions, Instantly,与Visionary ┃
┃ ┃
┃ --- ┃
┃ ┃
┃ Hi [Name], ┃
┃ ┃
┃ Is你的team drowning在data但starving为wisdom? ┃
┃ ┃
┃ Introducing **Visionary**,我们的groundbreaking AI-powered CRM ┃
┃ 那个doesn't just store customer information—it understands it. ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs,和flags at-risk accounts, empowering你的team来act ┃
┃ proactively. ┃
┃ ┃
┃ Ready来transform你的customer relationships? ┃
┃ ┃
┃ **[Schedule 一个15-Minute Live 演示]** ┃
┃ ┃
┃ All best, ┃
┃ Visionary Team ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
 最终 评分: 9/10 

### 结果分析

这detailed implementation reveals 一个two-layered improvement process:

1. **Intra-Task Improvement (Self-Refine):** 期间 first run, 代理's 初始草稿 是deliberately mediocre (评分: 4/10). logs clearly显示specific, actionable feedback从Senior Editor. 代理 then produced 一个revised draft 那个was 一个dramatic improvement, earning 一个9/10和getting approved. 这shows immediate benefit的loop为improving 一个single output.

2. **Inter-Task Improvement (Persistent Learning):** 在second demonstration, 代理 是given 一个*new* task. 然而，它的生成器现在配备了一个高质量的示例从previous successful run.输出log 是proof的learning: 代理's **first draft为new product 是so好的那个it 评分d 一个9/10 immediately**, requiring no revisions. 这是 一个强大的 demonstration的learning. 代理's baseline性能has improved because它learned从its past, editor-approved successes.

这second part 是一个direct, practical analogy为RLHF. 我们是 reinforcing 代理's behavior通过showing它examples的what "good" looks like, thereby improving它的ability来generate high-quality outputs从very first attempt在future 任务.

## 总结

在这个notebook中，我们实现了 一个comprehensive和复杂的 **Self-Improvement Loop**. 我们已经 demonstrated 那个这个architecture 是not just 关于refining 一个single piece的work，但is 一个强大的 paradigm为creating 代理那个genuinely learn和improve over time. 通过separating 角色提供动力的**generator**和**critic**，我们创建一个dynamic的feedback和revision 那个consistently elevates quality的代理's output. 通过adding 一个**persistent memory**为high-quality results，我们创建一个positive feedback loop 那个improves 代理's baseline capabilities, making它more 高效的和有效的在future 任务.

虽然risk的critic reinforcing它的own biases 被重新al和requires careful management, potential来构建agents 那个learn从experience 是一个transformative步骤towards more autonomous, capable,和intelligent AI 系统。